In [1]:
# recent gameweek
latest_gameweek = 14

In [2]:
import pandas as pd
import numpy as np

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# fetch FPL data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + '.csv'
players = pd.read_csv(filepath, index_col=0)

In [4]:
# how many gameweeks do we look form for
for i in [5,10]:
    number_gameweeks = i
    # create a list of relevant column names
    column_list = []
    for i in range(0,number_gameweeks):
        column_list.append('xPoints week ' + str(latest_gameweek-i))
    # calculate form
    column_name = 'form ' + str(number_gameweeks)
    players[column_name] = players[column_list].mean(axis=1)

In [5]:
# give a sorted list showing the players with best 'form 5'
players[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game',\
                                'form 5', 'form 10']].sort_values(by='form 5', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game,form 5,form 10
id,,,,,,,,
272,De Bruyne,12.037037,65,5.4,83.025269,6.897484,9.007967,7.092806
134,Ward,6.000000,15,2.5,16.443572,2.740595,8.181345,5.583866
40,Trézéguet,8.928571,25,2.8,40.150208,4.496823,7.006531,4.413779
302,Fernandes,13.076923,102,7.8,87.041583,6.656121,6.842947,7.384685
224,Vardy,12.948718,101,7.8,90.556854,6.993500,6.729976,7.325269
276,Sterling,12.037037,65,5.4,61.254033,5.088797,6.698223,5.184346
254,Salah,12.967033,118,9.1,91.945624,7.090722,6.620020,6.718664
275,Mahrez,10.000000,53,5.3,50.145312,5.014531,6.589233,5.184143
150,Sigurdsson,13.214286,37,2.8,43.904935,3.322536,6.447120,3.853455


In [6]:
# save data
filepath = '../data/fpl/data_week' + str(latest_gameweek) + '.csv'
players.to_csv(filepath)